In [1]:
from pyspark.sql import SparkSession
MAX_MEMORY = '8g'
spark = SparkSession.builder.appName('taxi-fare-prediction_2nd')\
                            .config('spark.driver.memory', MAX_MEMORY)\
                            .config('spark.executor.memory', MAX_MEMORY)\
                            .getOrCreate()
spark

In [2]:
import os 
cwd = os.getcwd()
trip_data_path = os.path.join(cwd, 'learning_spark_data', 'trips', '*.csv')
trip_data_path


'/home/jovyan/work/learning_spark_data/trips/*.csv'

In [3]:
file_path = f"file:///{trip_data_path.replace(os.sep,'/')}"
file_path
trip_df = spark.read.csv(file_path, inferSchema=True, header=True)
trip_df.printSchema()


root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [4]:
trip_df.createOrReplaceTempView('trips') # 테이블 생성 

In [5]:
query = """
SELECT 
    passenger_count,
    PULocationID as pickup_location_id,
    DOLocationID as dropoff_location_id,
    trip_distance,
    HOUR(tpep_pickup_datetime) as pickup_time,
    DATE_FORMAT(TO_DATE(tpep_pickup_datetime), 'EEEE') AS day_of_week,
    total_amount
FROM
    trips
WHERE
    total_amount < 5000
    AND total_amount > 0
    AND trip_distance > 0
    AND trip_distance < 500
    AND passenger_count < 4
    AND TO_DATE(tpep_pickup_datetime) >= '2021-01-01'
    AND TO_DATE(tpep_pickup_datetime) < '2021-08-01'
"""


In [6]:
data_df = spark.sql(query) # data_df 사용할거임
data_df.show(5)

+---------------+------------------+-------------------+-------------+-----------+-----------+------------+
|passenger_count|pickup_location_id|dropoff_location_id|trip_distance|pickup_time|day_of_week|total_amount|
+---------------+------------------+-------------------+-------------+-----------+-----------+------------+
|              0|               138|                265|         16.5|          0|     Monday|       70.07|
|              1|                68|                264|         1.13|          0|     Monday|       11.16|
|              1|               239|                262|         2.68|          0|     Monday|       18.59|
|              1|               186|                 91|         12.4|          0|     Monday|        43.8|
|              2|               132|                265|          9.7|          0|     Monday|        32.3|
+---------------+------------------+-------------------+-------------+-----------+-----------+------------+
only showing top 5 rows



In [7]:
# train,test split 8:2, seed 1
train_df, test_df = data_df.randomSplit([0.8, 0.2], seed=1)
train_df.show(5)

+---------------+------------------+-------------------+-------------+-----------+-----------+------------+
|passenger_count|pickup_location_id|dropoff_location_id|trip_distance|pickup_time|day_of_week|total_amount|
+---------------+------------------+-------------------+-------------+-----------+-----------+------------+
|              0|                 4|                  4|          0.1|         18|   Saturday|         6.3|
|              0|                 4|                 79|          0.7|         12|    Tuesday|         8.8|
|              0|                 4|                 79|          0.7|         23|   Saturday|       12.35|
|              0|                 4|                 79|          0.9|         14|     Monday|         9.8|
|              0|                 4|                114|          0.9|         18|     Friday|        10.8|
+---------------+------------------+-------------------+-------------+-----------+-----------+------------+
only showing top 5 rows



# 파이프라인 생성
- 전처리 과정을 각 스테이지로 정의해서 쌓는다
- 범주형: one-hot encoding: 'pickup_location_id', 'dropoff_location_id', 'day_of_week'
- 수치형: StandardScaler: 'passenger_count', 'trip_distance', 'pickup_time'                   

In [8]:
stages = []

In [9]:
# 전처리 파이프라인
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline


In [10]:
cat_features = ['pickup_location_id', 'dropoff_location_id', 'day_of_week']
for cat in cat_features:
        cat_index = StringIndexer(inputCol=cat, outputCol=cat+'_idx').setHandleInvalid('keep')
        onehot_endcode = OneHotEncoder(inputCols=[cat_index.getOutputCol()], #_idx col
                                      outputCols=[cat+'_onehot']) #postfix
        stages += [cat_index, onehot_endcode] #collist
stages

[StringIndexer_ac94f14e6724,
 OneHotEncoder_41e5c72a3d61,
 StringIndexer_b5d07b88d404,
 OneHotEncoder_b6a901939ba6,
 StringIndexer_aa464b41df88,
 OneHotEncoder_a3d3eef7cb06]

In [11]:
from pyspark.ml.feature import StandardScaler, VectorAssembler
num_features = ['passenger_count', 'trip_distance', 'pickup_time']

for num in num_features:
    num_assembler = VectorAssembler(inputCols=[num], outputCol=num+'_vector')
    num_scaler = StandardScaler(inputCol=num_assembler.getOutputCol(), outputCol=num+'_scaled')
    stages += [num_assembler, num_scaler]
stages

[StringIndexer_ac94f14e6724,
 OneHotEncoder_41e5c72a3d61,
 StringIndexer_b5d07b88d404,
 OneHotEncoder_b6a901939ba6,
 StringIndexer_aa464b41df88,
 OneHotEncoder_a3d3eef7cb06,
 VectorAssembler_a10081217617,
 StandardScaler_3eb8c478d2a3,
 VectorAssembler_c5a044e0b904,
 StandardScaler_19dc2c759a93,
 VectorAssembler_8df8b522a3ee,
 StandardScaler_6c208456bf5b]

In [12]:
assembler_input = [cat+'_onehot' for cat in cat_features] + [num+'_scaled' for num in num_features]
assembler_input

['pickup_location_id_onehot',
 'dropoff_location_id_onehot',
 'day_of_week_onehot',
 'passenger_count_scaled',
 'trip_distance_scaled',
 'pickup_time_scaled']

In [13]:
assembler = VectorAssembler(inputCols=assembler_input, outputCol='feature_vector')
stages += [assembler]
stages

[StringIndexer_ac94f14e6724,
 OneHotEncoder_41e5c72a3d61,
 StringIndexer_b5d07b88d404,
 OneHotEncoder_b6a901939ba6,
 StringIndexer_aa464b41df88,
 OneHotEncoder_a3d3eef7cb06,
 VectorAssembler_a10081217617,
 StandardScaler_3eb8c478d2a3,
 VectorAssembler_c5a044e0b904,
 StandardScaler_19dc2c759a93,
 VectorAssembler_8df8b522a3ee,
 StandardScaler_6c208456bf5b,
 VectorAssembler_dd6b4243e269]

In [14]:
# 하나로 뭉쳐서 ??????


In [15]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=stages)
fitted_transform = pipeline.fit(train_df)
# fit 하면 학습함 
vtrain_df = fitted_transform.transform(train_df)
vtrain_df.printSchema()

root
 |-- passenger_count: integer (nullable = true)
 |-- pickup_location_id: integer (nullable = true)
 |-- dropoff_location_id: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- pickup_location_id_idx: double (nullable = false)
 |-- pickup_location_id_onehot: vector (nullable = true)
 |-- dropoff_location_id_idx: double (nullable = false)
 |-- dropoff_location_id_onehot: vector (nullable = true)
 |-- day_of_week_idx: double (nullable = false)
 |-- day_of_week_onehot: vector (nullable = true)
 |-- passenger_count_vector: vector (nullable = true)
 |-- passenger_count_scaled: vector (nullable = true)
 |-- trip_distance_vector: vector (nullable = true)
 |-- trip_distance_scaled: vector (nullable = true)
 |-- pickup_time_vector: vector (nullable = true)
 |-- pickup_time_scaled: vector (nullable = true)
 |-- feature_vector: vector (nul

In [16]:
vtrain_df.select('feature_vector').show(2)

+--------------------+
|      feature_vector|
+--------------------+
|(533,[62,311,527,...|
|(533,[62,280,526,...|
+--------------------+
only showing top 2 rows



In [17]:
from pyspark.ml.regression import LinearRegression

# LinearRgression 생성 maxIter=50, labelCol='total_amount', featuresCol='features'
lr = LinearRegression(maxIter=50, labelCol='total_amount', featuresCol='feature_vector')
# fit
model = lr.fit(vtrain_df) # 천만건 데이터학습 



In [18]:
# vassembler.transform(test)
vtest_df = fitted_transform.transform(test_df)
# 테스트 데이터로 예측
pred = model.transform(vtest_df)

In [19]:
# pred.cache()

In [20]:
pred.select('total_amount', 'prediction').show(3)

+------------+------------------+
|total_amount|        prediction|
+------------+------------------+
|       10.55|12.695522792729275|
|        13.3|14.450558014776915|
|        21.3|21.108271361254218|
+------------+------------------+
only showing top 3 rows



In [21]:
model.summary.r2, model.summary.rootMeanSquaredError

(0.80849012500813, 5.6485201652667625)

In [22]:
spark.stop()